In [1]:
### importing modules and simulator class
%matplotlib inline

import autograd as ag
import autograd.numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from numpy.random import choice

class Simulator:

    """ Base class for simulators with access to joint score and joint likelihood ratios. """

    def __init__(self):
        pass

    def get_discretization(self):
        raise NotImplementedError

    def theta_defaults(self, n_thetas=100, single_theta=False, random=True):
        raise NotImplementedError()

    def rvs(self, theta, n, random_state=None):
        raise NotImplementedError()

    def rvs_score(self, theta, theta_score, n, random_state=None):
        raise NotImplementedError()

    def rvs_ratio(self, theta, theta0, theta1, n, random_state=None):
        raise NotImplementedError()

    def rvs_ratio_score(self, theta, theta0, theta1, theta_score, n, random_state=None):
        raise NotImplementedError()

In [2]:
def random_numbers():
    rng = np.random.rand(6)
    return(np.array(rng))
rng = random_numbers()


def make_weights(theta):
    p_roll = []
    p_i = 1/6 * rng**(theta)
    p_roll += [p_i]
    p_roll = np.array(p_roll)
    p_roll = p_roll/np.sum(p_roll)
    return(np.array(p_roll)[0])

In [100]:
CHUTES_LADDERS = {1:38, 4:14, 9:31, 16:6, 21:42, 28:84, 36:44,
                  47:26, 49:11, 51:67, 56:53, 62:19, 64:60,
                  71:91, 80:100, 87:24, 93:73, 95:75, 98:78} #dictionary of chutes and ladders



class Chutes_Ladders(Simulator):
    n=1
    ### initialization                    
    def __init__(self):
        self.d_simulate = ag.grad_and_aux(self.grad_simulate)
    
    
    ### creates samples x given theta (weights); p(x|theta)
    def simulate(self, theta=0, random_state=None): 
        p_i = make_weights(theta)
        
        turns_list = [] #number of turns per game
        winner = [] #which player wins each game
        rolls_list = []
        log_list = []

        position1_list = [] #player 1 position list
        position2_list = [] #player 2 position list
        final_position = []

   
        counter = 0
        prob_side = []

        log_p_xz = 1.0
        roll_list = []
        position = []
        position1 = 0
        position2 = 0
        turns = 0
        counter += 1                
        while position1 < 100 and position2 < 100:
            turns += 1
            random_number1 = np.random.rand()
            random_number2 = np.random.rand()
        ### player_1's turn
            if random_number1 <= p_i[0]:
                roll1 = 1
                log_p_xz *= p_i[0]
            elif random_number1 <= p_i[0] + p_i[1]:
                roll1 = 2
                log_p_xz *= p_i[1]
            elif random_number1 <= p_i[0] + p_i[1] + p_i[2]:
                roll1 = 3
                log_p_xz *= p_i[2]
            elif random_number1 <= p_i[0] + p_i[1] + p_i[2] + p_i[3]:
                roll1 = 4
                log_p_xz *= p_i[3]
            elif random_number1 <= p_i[0] + p_i[1] + p_i[2] + p_i[3]+ p_i[4]:
                roll1 = 5
                log_p_xz *= p_i[4]
            else:
                roll1 = 6
                log_p_xz *= p_i[5]

            position1 += roll1
            position1 = CHUTES_LADDERS.get(position1, position1)

            ### player_2's turn
            if random_number2 <= p_i[0]:
                roll2 = 1
                log_p_xz *= p_i[0]
            elif random_number2 <= p_i[0] + p_i[1]:
                roll2 = 2
                log_p_xz *= p_i[1]
            elif random_number2 <= p_i[0] + p_i[1] + p_i[2]:
                roll2 = 3
                log_p_xz *= p_i[2]
            elif random_number2 <= p_i[0] + p_i[1] + p_i[2] + p_i[3]:
                roll2 = 4
                log_p_xz *= p_i[3]
            elif random_number2 <= p_i[0] + p_i[1] + p_i[2] + p_i[3]+ p_i[4]:
                roll2 = 5
                log_p_xz *= p_i[4]
            else:
                roll2 = 6
                log_p_xz *= p_i[5]

            position2 += roll2
            position2 = CHUTES_LADDERS.get(position2, position2)

            ### logging positions/roll data
            position1_list += [position1]
            position2_list += [position2]
            position += [[position1, position2]]
            roll_list += [[roll1, roll2]]



            ### ends game
            if position1 >= 100:
                final_position += [position]
                winner += [0.0]
                turns_list += [turns]
                rolls_list += [roll_list]
                log_p_xz = (np.log(log_p_xz))
                prob_side.append(log_p_xz)


                position1_list += ["Player 1 Wins"]
                position2_list += ["Player 1 Wins"]
                continue

            elif position2 >= 100:
                final_position += [position]
                rolls_list += [roll_list]
                winner += [1.0] 
                turns_list += [turns]     
                log_p_xz = (np.log(log_p_xz))
                prob_side.append(log_p_xz)

                position1_list += ["Player 2 Wins"]
                position2_list += ["Player 2 Wins"]
                continue
            log_list.append(prob_side)                    
        return(log_p_xz, turns, rolls_list)

    def grad_simulate(self, theta=0, n=1, random_state=None): 
        p_i = make_weights(theta)
        
        turns_list = [] #number of turns per game
        winner = [] #which player wins each game
        rolls_list = []
        log_list = []

        position1_list = [] #player 1 position list
        position2_list = [] #player 2 position list
        final_position = []

   
        counter = 0
        prob_side = []

        log_p_xz = 1.0
        roll_list = []
        position = []
        position1 = 0
        position2 = 0
        turns = 0
        counter += 1                
        while position1 < 100 and position2 < 100:
            turns += 1
            random_number1 = np.random.rand()
            random_number2 = np.random.rand()
        ### player_1's turn
            if random_number1 <= p_i[0]:
                roll1 = 1
                log_p_xz *= p_i[0]
            elif random_number1 <= p_i[0] + p_i[1]:
                roll1 = 2
                log_p_xz *= p_i[1]
            elif random_number1 <= p_i[0] + p_i[1] + p_i[2]:
                roll1 = 3
                log_p_xz *= p_i[2]
            elif random_number1 <= p_i[0] + p_i[1] + p_i[2] + p_i[3]:
                roll1 = 4
                log_p_xz *= p_i[3]
            elif random_number1 <= p_i[0] + p_i[1] + p_i[2] + p_i[3]+ p_i[4]:
                roll1 = 5
                log_p_xz *= p_i[4]
            else:
                roll1 = 6
                log_p_xz *= p_i[5]

            position1 += roll1
            position1 = CHUTES_LADDERS.get(position1, position1)

            ### player_2's turn
            if random_number2 <= p_i[0]:
                roll2 = 1
                log_p_xz *= p_i[0]
            elif random_number2 <= p_i[0] + p_i[1]:
                roll2 = 2
                log_p_xz *= p_i[1]
            elif random_number2 <= p_i[0] + p_i[1] + p_i[2]:
                roll2 = 3
                log_p_xz *= p_i[2]
            elif random_number2 <= p_i[0] + p_i[1] + p_i[2] + p_i[3]:
                roll2 = 4
                log_p_xz *= p_i[3]
            elif random_number2 <= p_i[0] + p_i[1] + p_i[2] + p_i[3]+ p_i[4]:
                roll2 = 5
                log_p_xz *= p_i[4]
            else:
                roll2 = 6
                log_p_xz *= p_i[5]

            position2 += roll2
            position2 = CHUTES_LADDERS.get(position2, position2)

            ### logging positions/roll data
            position1_list += [position1]
            position2_list += [position2]
            position += [[position1, position2]]
            roll_list += [[roll1, roll2]]



            ### ends game
            if position1 >= 100:
                final_position += [position]
                winner += [0.0]
                turns_list += [turns]
                rolls_list += [roll_list]
                log_p_xz = (np.log(log_p_xz))
                prob_side.append(log_p_xz)


                position1_list += ["Player 1 Wins"]
                position2_list += ["Player 1 Wins"]
                continue

            elif position2 >= 100:
                final_position += [position]
                rolls_list += [roll_list]
                winner += [1.0] 
                turns_list += [turns]     
                log_p_xz = (np.log(log_p_xz))
                prob_side.append(log_p_xz)

                position1_list += ["Player 2 Wins"]
                position2_list += ["Player 2 Wins"]
                continue
            log_list.append(prob_side)                    
        return(log_p_xz, turns)

    
    def get_theta_n(self, n_theta): #used to get theta values for rvs_ratio, if theta is not specified
        all_theta = []
        for i in range(n_theta):
            x = np.random.dirichlet(np.ones(6), size=1)
            all_theta += [x.tolist()[0]]
        return(all_theta)
    

    
    
    
    def rvs_test(self, theta, n, random_state=None): 
        turns_list = []
        log_list = []
        for i in range(n):
            log_p_xz, turns, rolls_list = self.simulate(theta)
            turns_list.append(turns)
            log_list.append(log_p_xz)
        log_list = np.array(log_list)
        return(log_list, turns_list)

    def rvs(self, theta_list, n, random_state = None): #finalized
        turns = []
        for theta in theta_list:
            result = test.rvs_test(theta, n=n)
            turns.append(result)
        return(turns)
    
    
    

    def rvs_score_test(self, theta, n, random_state=None):
        all_x = []
        all_t_xz = []
        
        for i in range(n):
            _, x = self.grad_simulate(theta)
            t_xz, _ = self.d_simulate(theta)

            all_x.append(x)
            all_t_xz.append(t_xz)
        return(all_t_xz, all_x)    

    def rvs_score(self, theta_list, n, random_state=None): #finalized
        score = []
        turns = []
        for theta in theta_list:
            t_xz, all_x = self.rvs_score_test(theta, n=n)
            score.append(t_xz)
            turns.append(all_x)
        return(score, turns)

    
    
    
    
    
    def rvs_ratio(self, theta0, theta_list, n=1, random_state=None):
        log_p0_xz, turns_0, rolls_0 = self.simulate(theta0)
        
        roll_array = []
        total_rolls = [roll for pair in rolls_0[0] for roll in pair]
        weights = []
        for theta in theta_list:
            weights.append(make_weights(theta))
        for weight in weights:
            theta_n_list = []
            for i in range(n):
                counter = 1.0
                for roll in total_rolls:
                    counter *= weight[roll-1]
                theta_n_list.append(counter)
            
            roll_array.append(log_p0_xz/np.log(np.array(theta_n_list)))
        return(roll_array, turns_0)


    
    
    

    def rvs_ratio_score(self, theta0, theta_list, n, random_state=None):
        ratio,_ = self.rvs_ratio(theta0 = theta0, theta_list = theta_list, n=n)
        t_xz, turns = self.rvs_score(theta_list=theta_list, n=n)
        return(turns, t_xz, ratio)        


test = Chutes_Ladders()

In [75]:
#rvs  #number of lists = number of thetas; length of each list = number of trials
test.rvs(theta_list = [0,.1,.2], n=5) #returns log probability and number of turns

[(array([ -93.1714924 , -125.42316285, -132.59020072, -118.25612497,
          -71.67037877]), [26, 35, 37, 33, 20]),
 (array([-86.48767056, -85.23762858, -88.61257028, -47.09147303, -64.95624851]),
  [24, 24, 25, 13, 18]),
 (array([ -60.1882165 ,  -77.66078976, -103.18240579,  -86.4315197 ,
          -43.47505354]), [17, 22, 29, 24, 12])]

In [5]:
#rvs score  #number of lists = number of thetas; length of each list = number of trials
test.rvs_score(theta_list=[.3,.4,.5], n=5) #returns score and number of turns


([[-0.35356929872258025,
   1.7601360700356214,
   2.8508000273331056,
   2.7299644773790903,
   -5.2377022099390764],
  [-0.87638065802674747,
   -0.72722333823730745,
   -7.5635447468563468,
   -2.0989719778642799,
   7.1877375935082979],
  [-0.95288062883940705,
   -2.7250497062776833,
   -0.094607947829898453,
   -2.5262142136459462,
   3.5255360990572004]],
 [[35, 38, 39, 61, 27], [12, 28, 45, 14, 26], [12, 15, 26, 39, 13]])

In [103]:
#rvs ratio
test.rvs_ratio(theta0=.3, theta_list=[3,2,0,.3]) #returns ratios and number of turns

([array([ 0.70516268]),
  array([ 0.82729662]),
  array([ 0.98737589]),
  array([ 1.])],
 11)

In [101]:
#rvs ratio score
test.rvs_ratio_score(theta0 = [.3], theta_list = [.3, .2], n=5) #theta0 is a list
#returns number of turns scores and ratios

([[8, 34, 40, 20, 18], [18, 18, 19, 11, 13]],
 [[0.27513846015735011,
   8.3996457898797523,
   0.01097005388563721,
   1.3731781040349196,
   2.5331766203064521],
  [2.1389752621207601,
   4.36555939123968,
   -7.4335637180002108,
   -3.7095477324703907,
   0.13896580030658578]],
 [array([ 1.,  1.,  1.,  1.,  1.]),
  array([ 1.01035006,  1.01035006,  1.01035006,  1.01035006,  1.01035006])])